# 2次元のIsing modelをTensor renormalization groupを用いて計算

ハミルトニアンが次の式で与えられる正方格子上の2次元 Ising modelを計算します。
 $$ \mathcal{H} = -J \sum_{\langle i,j\rangle} \sigma_i \sigma_j$$
ここで $\sigma_i = \pm 1$

#### パラメータ
* n: 系の大きさ$L = 2^n$を指定するパラメータ
* D: ボンド次元

In [ ]:
#ライブラリのインストール
!pip install numpy
!pip install matplotlib
!pip install scipy
!pip install numba

In [ ]:
#ライブラリやモジュールの読み込み
import numpy as np
try:
    import TRG_library
except ImportError:
    !wget https://raw.githubusercontent.com/888ten/Ising_TRG/main/TRG_library.py
    import TRG_library
import os
import pickle
from matplotlib import pyplot

In [ ]:
#####パラメータを設定
Tc = 2.0 / np.log(1.0 + np.sqrt(2.0))
n = 10 ## L = 2^n
T_min = 2.0 ##最低温度
T_max = 2.7 ##最高温度
T_step = 0.01

T_list = np.arange(T_min, T_max, T_step)
D = 24 ##ボンド次元
L = 2 ** n
TRG_step = 2 * n - 1
#####

data_file = f"trgdata_ex1-1_D{D}_L{L}.dat"

def Calculate_EC(T, f):
    T_num = len(T)
    E = np.empty(T_num - 2)
    C = np.empty(T_num - 2)
    T_cut = np.empty(T_num - 2)
    for i in range(T_num - 2):
        E[i] = f[i + 1] - T[i + 1] * (f[i + 2] - f[i]) / (T[i + 2] - T[i])
        C[i] = -T[i + 1] * (f[i + 2] + f[i] - 2.0 * f[i + 1]) / (T[i + 2] - T[i + 1]) ** 2
        T_cut[i] = T[i + 1]
    return T_cut, E, C

def read_free_energy(file_name):
    T = []
    f = []
    for line in open(file_name, "r"):
        data = line.split()
        if data[0] == "#":
            header = line
            continue
        T.append(float(data[4]))
        f.append(float(data[6]))
    return T, f

# TRGシミュレーション
free_energy_density = []
for T in T_list:
    free_energy_density.append(TRG_library.TRG_Square_Ising(T, D, TRG_step))

filename_exact = f"exact_output/free_energy_exact_L{L}.dat"
if not os.path.exists(filename_exact):
    !wget https://raw.githubusercontent.com/888ten/Ising_TRG/main/{filename_exact} -P exact_output

T_e, f_e = read_free_energy(filename_exact)

T_cut, E, C = Calculate_EC(T_list, free_energy_density)

T_cut_e, E_e, C_e = Calculate_EC(T_e, f_e)

# データ保存
with open(data_file, "wb") as f:
    obs_list = [free_energy_density, E, C, T_list]
    pickle.dump(obs_list, f)

In [ ]:
pyplot.figure()
pyplot.title("Free energy")
pyplot.xlabel("$T$")
pyplot.ylabel("$f$")
pyplot.plot(T_list, free_energy_density, "o", label=f"L={L}: TRG", color="purple", markersize=3)
pyplot.plot(T_e, f_e, "-", label="exact", color="green")
pyplot.legend()

In [ ]:
result = []
for x, y in zip(f_e,free_energy_density):
    relative_error = abs(x - y) / abs(x)  # 相対誤差の計算
    result.append(relative_error)

In [ ]:
pyplot.figure()
pyplot.title("relative error")
pyplot.xlabel("$T$")
pyplot.ylabel("$relative error$")
pyplot.plot(T_list,result,"o",label = "relative error of free energy", color = "purple")
pyplot.legend()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.title("relative error")
plt.xlabel("$T$")
plt.ylabel("$relative error$")
plt.xlim(2.1,2.5)
plt.ylim(10**(-12), 10**(-5.5))
plt.semilogy(T_list, result, "-", label = "relative error of free energy", color = "purple")
plt.legend()
plt.show()

In [ ]:
pyplot.figure()
pyplot.title("Energy")
pyplot.xlabel("$T$")
pyplot.ylabel("$E$")
pyplot.plot(T_cut, E, "o", label=f"L={L}", color="purple", markersize=3)
pyplot.plot(T_cut_e, E_e, "-", label="exact", color="green")
pyplot.legend()

In [ ]:
pyplot.figure()
pyplot.title("Specific heat")
pyplot.xlabel("$T$")
pyplot.ylabel("$C$")
pyplot.plot(T_cut, C, "o", label=f"L={L}", color="purple", markersize=3)
pyplot.plot(T_cut_e, C_e, "-", label="exact", color="green")
pyplot.legend()